In [2]:
import numpy as np
import pandas as pd

# Visualizastion
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone
from sklearn.ensemble import VotingClassifier

In [3]:
df_train = pd.read_csv("train.csv", index_col='id')
df_test = pd.read_csv("test.csv", index_col='id')

In [4]:
df_train.head()

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
id,,,,,,,,
0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18524 entries, 0 to 18523
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Time_spent_Alone           17334 non-null  float64
 1   Stage_fear                 16631 non-null  object 
 2   Social_event_attendance    17344 non-null  float64
 3   Going_outside              17058 non-null  float64
 4   Drained_after_socializing  17375 non-null  object 
 5   Friends_circle_size        17470 non-null  float64
 6   Post_frequency             17260 non-null  float64
 7   Personality                18524 non-null  object 
dtypes: float64(5), object(3)
memory usage: 1.3+ MB


# Preprocessing

## Impute Null Values

Imputing will be according to personality. For example, an introvert will likely have stage fear, and will be drained after sociaslizing. 
As for the numeric columns, we'll randomize low numbers in a reasonable interval which will be determined according to the dataset.

### Stage_fear and Drained_after_socializing 

# Visualization

# Feature Engineering

# Prediction

# Hyperparameter Optimization

# Run Optimal Model